In [ ]:
from selenium import webdriver 
from bs4 import BeautifulSoup
import re
import os, sys
import csv


In [ ]:
driver = webdriver.Chrome('C:/Users/Яна/chromedriver')

In [ ]:
urls=[]
links=[]
for i in range(1,159,1):
    list_mov = 'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&foot_min=45&metro%5B0%5D='+str(i)+'&offer_type=flat&only_foot=2'
    for j in range(1,30,1):
        page_mov = list_mov +'&p='+str(j)+'&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
        urls.append(page_mov)
        
for url in urls:
    driver.get(url)
    html = driver.page_source
    s = BeautifulSoup(html, 'lxml')
    items = s.find_all('a', class_='c6e8ba5398--header--1fV2A')
    for i in items:
        link = i.attrs['href']
        links.append(link)
        
for link in links:
    driver.get(link)
    html = driver.page_source
    a = BeautifulSoup(html, 'lxml')
    filename = a.title.string
    v=re.findall('([\d]+$)', filename)
    with open(f'C:/Users/Яна/pars1/{v}.txt', 'w', encoding='utf-8') as file:
        file.write(html)
        

path = "C:/Users/Яна/pars1"
dirs = os.listdir( path )
result1=[]
for file in dirs:
    with open(f'C:/Users/Яна/pars1/{file}', 'r', encoding= 'utf8') as file:
        for line in file:
            contents = file.read()
            soup = BeautifulSoup(contents, 'lxml')
            def get_room(soup):
                room=soup.find('h1', class_='a10a3f92e9--title--2Widg').text
                if room is not None:
                    return re.search(r'(^[\w|\s|-]+)', room).group(0)
                return None
            def get_address(soup):
                address=soup.find_all('a', class_='a10a3f92e9--underground_link--AzxRC')
                if address:
                    match=address[0]
                    match=match.string.replace('⋅', '').replace('   ', '')
                    return match
                else: 
                    match=None
                    return match
            def get_time(soup):
                time=soup.find_all('span', class_='a10a3f92e9--underground_time--1fKft')
                if time:
                    match1=time[0]
                    match1=match1.string.replace('⋅', '').replace('   ', '')
                    return match1
                else: 
                    match1=None
                    return match1
            def get_floor(soup):
                for a_ in soup.find_all('div', class_="a10a3f92e9--info--3pFQf"):
                    res=re.search(".*Этаж.*", a_.text)
                    if res is not None:
                        floor=a_.find('div', class_="a10a3f92e9--info-value--3ChCO").text
                        return re.match(r'(\d{1,3}){1}',floor).group(0)
                return None
            def get_type_home(soup):
                for a_ in soup.find_all('li', class_='a10a3f92e9--item--_ipjK'):
                    res1=re.search(".*Тип жилья.*", a_.text)
                    if res1 is not None:
                        return a_.find('span', class_="a10a3f92e9--value--3Ftu5").text
                return None
            def get_square(soup):
                square=soup.find('h1', class_='a10a3f92e9--title--2Widg').text
                if square is not None:
                    return re.search(r'(\d{2,3})(,\d{1,3})*',  square).group(0)
                return None
            def get_repair(soup):
                for a_ in soup.find_all('li', class_='a10a3f92e9--item--_ipjK'):
                    res2=re.search(".*Ремонт.*", a_.text)
                    if res2 is not None:
                        return a_.find('span', class_="a10a3f92e9--value--3Ftu5").text
                return None
            def get_year(soup):
                for a_ in soup.find_all('div', class_='a10a3f92e9--item--2Ig2y'):
                    res3=re.search(".*Год постройки.*", a_.text)
                    if res3 is not None:
                        return a_.find('div', class_="a10a3f92e9--value--38caj").text
                return None
            def get_time_new(soup):
                for a_ in soup.find_all('div', class_="a10a3f92e9--info--3pFQf"):
                    res4=re.search(".*Срок сдачи.*", a_.text)
                    if res4 is not None:
                        time=a_.find('div', class_="a10a3f92e9--info-value--3ChCO").text
                        return re.search(r'\d{4}',time).group(0)
                return None
            def get_house(soup):
                for a_ in soup.find_all('div', class_='a10a3f92e9--item--2Ig2y'):
                    res4=re.search(".*Тип дома.*", a_.text)
                    if res4 is not None:
                        return a_.find('div', class_="a10a3f92e9--value--38caj").text
                return None
            def get_price(soup):
                match3=soup.find('span', class_='a10a3f92e9--price_value--1iPpd').text.replace('₽', '')
                if match3 is not None:
                    return match3
                return None
            head=['Тип жилья','Кол-во комнат','Станция метро',"Время в пути",'Этаж','Общая площадь комнат', "Наличие ремонта","Год постройки",'Срок сдачи',"Тип дома","Цена"]
            date=[get_type_home(soup),get_room(soup),get_address(soup),get_time(soup),get_floor(soup),get_square(soup),get_repair(soup),get_year(soup),get_time_new(soup),get_house(soup),get_price(soup)]
            result1.append(date)
with open('sw_data_new.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(head)
    for row in result1:
        writer.writerow(row)